In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
## DB 사전작업


In [ ]:
# ===== 0. Colab 환경 세팅 =====
from google.colab import drive
drive.mount('/content/drive')

# 필요 라이브러리 설치
!pip install -q --upgrade "torch>=2.3.0" "transformers>=4.40.0" "accelerate" "sentence-transformers"

import os
import json
import sqlite3
from collections import defaultdict

import numpy as np
from tqdm import tqdm

import torch
from sentence_transformers import SentenceTransformer

# BGE-M3 임베딩 모델
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder = SentenceTransformer("BAAI/bge-m3", device=device)

print("Using device:", device)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 60.8 MB/s eta 0:00

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
# ===== 1. 경로 설정 =====

SPIDER_ROOT = "/content/drive/MyDrive/ai_intensive2/spider_data"
DB_ROOT     = os.path.join(SPIDER_ROOT, "database")
TABLES_JSON = os.path.join(SPIDER_ROOT, "tables.json")

OUT_ROOT = os.path.join(SPIDER_ROOT, "preprocessed")
os.makedirs(OUT_ROOT, exist_ok=True)

TABLE_INDEX_JSONL  = os.path.join(OUT_ROOT, "table_index.jsonl")
TABLE_EMB_NPY      = os.path.join(OUT_ROOT, "table_index_emb.npy")

SCHEMA_INDEX_JSONL = os.path.join(OUT_ROOT, "schema_index.jsonl")
SCHEMA_EMB_NPY     = os.path.join(OUT_ROOT, "schema_index_emb.npy")

CELL_INDEX_JSONL   = os.path.join(OUT_ROOT, "cell_index.jsonl")
CELL_EMB_NPY       = os.path.join(OUT_ROOT, "cell_index_emb.npy")

print("SPIDER_ROOT:", SPIDER_ROOT)
print("DB_ROOT    :", DB_ROOT)
print("OUT_ROOT   :", OUT_ROOT)

# ===== tables.json 로드 =====
with open(TABLES_JSON, "r", encoding="utf-8") as f:
    tables_meta = json.load(f)

# db_id -> meta
db_meta = {db["db_id"]: db for db in tables_meta}
print("총 DB 개수:", len(db_meta))
print("예시 db_id:", list(db_meta.keys())[:5])


SPIDER_ROOT: /content/drive/MyDrive/ai_intensive2/spider_data
DB_ROOT    : /content/drive/MyDrive/ai_intensive2/spider_data/database
OUT_ROOT   : /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed
총 DB 개수: 166
예시 db_id: ['perpetrator', 'college_2', 'flight_company', 'icfp_1', 'body_builder']


In [ ]:
# ===== 2. db_schemas 구성 =====

def build_db_schema_info(db_meta):
    db_schemas = {}

    for db_id, meta in db_meta.items():
        table_names        = meta["table_names"]              # 정규화 이름
        table_names_orig   = meta["table_names_original"]     # 실제 sqlite 테이블명
        column_names       = meta["column_names"]             # (table_idx, col_name_norm)
        column_names_orig  = meta["column_names_original"]    # (table_idx, col_name_db)
        column_types       = meta["column_types"]
        primary_keys       = set(meta.get("primary_keys", []))
        foreign_keys       = meta.get("foreign_keys", [])

        # 테이블 정보
        tables = {}
        for tid, (t_norm, t_orig) in enumerate(zip(table_names, table_names_orig)):
            tables[tid] = {
                "table_name_norm": t_norm,
                "table_name_db":   t_orig
            }

        # PK/FK 마킹
        is_pk = {cid: False for cid in range(len(column_names))}
        is_fk = {cid: False for cid in range(len(column_names))}
        for cid in primary_keys:
            is_pk[cid] = True
        for (c1, c2) in foreign_keys:
            is_fk[c1] = True
            is_fk[c2] = True

        columns = {}
        columns_by_table = defaultdict(list)

        for cid, ((tidx, col_norm), (tidx2, col_orig)) in enumerate(
            zip(column_names, column_names_orig)
        ):
            # tidx == -1 은 pseudo-* 컬럼 → 무시
            if tidx < 0:
                continue

            if tidx != tidx2:
                print(f"[WARN] {db_id}: table_idx mismatch in column {cid}: {tidx} vs {tidx2}")

            tmeta = tables[tidx]
            col_type = column_types[cid]

            col_info = {
                "db_id": db_id,
                "table_idx": tidx,
                "table_name_norm": tmeta["table_name_norm"],
                "table_name_db":   tmeta["table_name_db"],
                "col_id": cid,
                "col_name_norm": col_norm,
                "col_name_db":   col_orig,
                "col_type": col_type,
                "is_pk": is_pk[cid],
                "is_fk": is_fk[cid],
            }
            columns[cid] = col_info
            columns_by_table[tidx].append(col_info)

        db_schemas[db_id] = {
            "tables": tables,
            "columns": columns,
            "columns_by_table": columns_by_table,
            "primary_keys": primary_keys,
            "foreign_keys": foreign_keys,
        }

    return db_schemas

db_schemas = build_db_schema_info(db_meta)
print("스키마 파싱 완료. 예시 DB:")
example_db = list(db_schemas.keys())[0]
print(example_db, "=> 테이블 수:", len(db_schemas[example_db]["tables"]))


스키마 파싱 완료. 예시 DB:
perpetrator => 테이블 수: 2


In [ ]:
# ===== 3. TableIndex 생성 =====

table_entries = []
table_texts   = []

for db_id, schema in tqdm(db_schemas.items(), desc="Building TableIndex"):
    tables = schema["tables"]
    columns_by_table = schema["columns_by_table"]

    for tidx, tmeta in tables.items():
        tname_norm = tmeta["table_name_norm"]
        tname_db   = tmeta["table_name_db"]
        cols = columns_by_table.get(tidx, [])

        col_desc_list = [f"{c['col_name_norm']} ({c['col_type']})" for c in cols]
        col_desc_str  = ", ".join(col_desc_list) if col_desc_list else "no_columns"

        text = (
            f"db: {db_id}; "
            f"table: {tname_norm}; "
            f"db_table_name: {tname_db}; "
            f"columns: {col_desc_str}"
        )

        entry = {
            "db_id": db_id,
            "table_idx": tidx,
            "table_name_norm": tname_norm,
            "table_name_db": tname_db,
            "columns": [c["col_name_norm"] for c in cols],
            "text": text,
        }

        table_entries.append(entry)
        table_texts.append(text)

print("TableIndex 엔트리 수:", len(table_entries))

table_emb = encoder.encode(
    table_texts,
    normalize_embeddings=True,
    convert_to_numpy=True,
    show_progress_bar=True
)
print("TableIndex embedding shape:", table_emb.shape)

with open(TABLE_INDEX_JSONL, "w", encoding="utf-8") as f:
    for e in table_entries:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

np.save(TABLE_EMB_NPY, table_emb)
print("TableIndex 저장 완료:", TABLE_INDEX_JSONL, TABLE_EMB_NPY)


Building TableIndex: 100%|██████████| 166/166 [00:00<00:00, 42833.25it/s]

TableIndex 엔트리 수: 876


Batches:   0%|          | 0/28 [00:00<?, ?it/s]

TableIndex embedding shape: (876, 1024)
TableIndex 저장 완료: /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/table_index.jsonl /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/table_index_emb.npy


In [ ]:
# ===== 4. Schema-DB (컬럼 인덱스) 생성 =====

schema_entries = []
schema_texts   = []

for db_id, schema in tqdm(db_schemas.items(), desc="Building Schema-DB"):
    columns = schema["columns"]

    for cid, c in columns.items():
        text = (
            f"db: {db_id}; "
            f"table: {c['table_name_norm']}; "
            f"db_table_name: {c['table_name_db']}; "
            f"column: {c['col_name_norm']}; "
            f"db_column_name: {c['col_name_db']}; "
            f"type: {c['col_type']}; "
        )
        if c["is_pk"]:
            text += "primary key; "
        if c["is_fk"]:
            text += "foreign key; "

        entry = {
            "db_id": db_id,
            "col_id": cid,
            "table_idx": c["table_idx"],
            "table_name_norm": c["table_name_norm"],
            "table_name_db": c["table_name_db"],
            "col_name_norm": c["col_name_norm"],
            "col_name_db": c["col_name_db"],
            "col_type": c["col_type"],
            "is_pk": c["is_pk"],
            "is_fk": c["is_fk"],
            "text": text,
        }
        schema_entries.append(entry)
        schema_texts.append(text)

print("Schema-DB 엔트리 수:", len(schema_entries))

schema_emb = encoder.encode(
    schema_texts,
    normalize_embeddings=True,
    convert_to_numpy=True,
    show_progress_bar=True
)
print("Schema-DB embedding shape:", schema_emb.shape)

with open(SCHEMA_INDEX_JSONL, "w", encoding="utf-8") as f:
    for e in schema_entries:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

np.save(SCHEMA_EMB_NPY, schema_emb)
print("Schema-DB 저장 완료:", SCHEMA_INDEX_JSONL, SCHEMA_EMB_NPY)


Building Schema-DB: 100%|██████████| 166/166 [00:00<00:00, 18345.66it/s]

Schema-DB 엔트리 수: 4503


Batches:   0%|          | 0/141 [00:00<?, ?it/s]

Schema-DB embedding shape: (4503, 1024)
Schema-DB 저장 완료: /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/schema_index.jsonl /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/schema_index_emb.npy


In [ ]:
# ===== 5. Cell-DB (셀 값 인덱스) 생성 =====

# TableRAG 스타일을 위해 전처리에서 크게 자르지 않고 가능한 한 많이 넣고,
# 나중에 질의 시점에서 encoding budget B로 셀을 선택해 사용할 예정.
MAX_DISTINCT_PER_COLUMN = None   # None이면 LIMIT 없음. 필요하면 1000 같은 값으로.
MAX_TOTAL_CELLS = None           # 전체 셀 엔트리 상한. None이면 제한 없음.

cell_entries = []
cell_texts   = []

def quote_ident(name: str) -> str:
    """
    SQLite 식별자(테이블/컬럼 이름)를 "..." 로 안전하게 감싸기.
    이름 안의 " 는 "" 로 이스케이프.
    """
    if name is None:
        name = ""
    escaped = name.replace('"', '""')
    return f'"{escaped}"'


for db_id, schema in tqdm(db_schemas.items(), desc="Building Cell-DB (per DB)"):
    db_dir = os.path.join(DB_ROOT, db_id)
    sqlite_path = os.path.join(db_dir, f"{db_id}.sqlite")
    if not os.path.exists(sqlite_path):
        # 혹시 확장자가 다를 수 있으므로 fallback
        candidates = [f for f in os.listdir(db_dir)
                      if f.endswith(".sqlite") or f.endswith(".db")]
        if not candidates:
            print(f"[WARN] No sqlite/db file found in {db_dir}, skip DB {db_id}")
            continue
        sqlite_path = os.path.join(db_dir, candidates[0])

    conn = sqlite3.connect(sqlite_path)
    cur  = conn.cursor()

    columns_by_table = schema["columns_by_table"]

    for tidx, cols in columns_by_table.items():
        if not cols:
            continue

        tname_norm = cols[0]["table_name_norm"]
        tname_db   = cols[0]["table_name_db"]
        table_quoted = quote_ident(tname_db)

        # 실제 DB에 존재하는 컬럼 목록 확인
        try:
            cur.execute(f'PRAGMA table_info({table_quoted})')
            pragma_rows = cur.fetchall()
            cols_in_db = {row[1] for row in pragma_rows}  # row[1] = column name
        except Exception as e:
            print(f"[WARN] {db_id}.{tname_db} PRAGMA table_info 실패:", e)
            continue

        for c in cols:
            col_db_name   = c["col_name_db"]   # sqlite 실제 컬럼명 (tables.json 기준)
            col_norm_name = c["col_name_norm"]
            col_type      = c["col_type"]

            # 실제 DB에 없는 컬럼이면 스킵 (Kaggle 버전에서는 이게 많았음)
            if col_db_name not in cols_in_db:
                print(f"[WARN] {db_id}.{tname_db}.{col_db_name} 실제 DB에 없음 → 스킵")
                continue

            # 전체 셀 상한 체크
            if (MAX_TOTAL_CELLS is not None) and (len(cell_entries) >= MAX_TOTAL_CELLS):
                print(f"[INFO] MAX_TOTAL_CELLS={MAX_TOTAL_CELLS} 도달, 나머지 생략.")
                break

            col_quoted = quote_ident(col_db_name)

            # DISTINCT 값 + 빈도 추출
            if MAX_DISTINCT_PER_COLUMN is None:
                limit_clause = ""
            else:
                limit_clause = f"LIMIT {MAX_DISTINCT_PER_COLUMN}"

            query = f"""
                SELECT {col_quoted}, COUNT(*) as freq
                FROM {table_quoted}
                GROUP BY {col_quoted}
                ORDER BY freq DESC
                {limit_clause}
            """
            try:
                cur.execute(query)
                rows = cur.fetchall()
            except Exception as e:
                print(f"[WARN] {db_id}.{tname_db}.{col_db_name} 쿼리 실패:", e)
                continue

            for value, freq in rows:
                if (MAX_TOTAL_CELLS is not None) and (len(cell_entries) >= MAX_TOTAL_CELLS):
                    break

                try:
                    v_str = str(value)
                except Exception:
                    v_str = repr(value)

                text = (
                    f"db: {db_id}; "
                    f"table: {tname_norm}; "
                    f"db_table_name: {tname_db}; "
                    f"column: {col_norm_name}; "
                    f"db_column_name: {col_db_name}; "
                    f"type: {col_type}; "
                    f"value: {v_str}; "
                    f"frequency: {freq}"
                )

                entry = {
                    "db_id": db_id,
                    "table_idx": tidx,
                    "table_name_norm": tname_norm,
                    "table_name_db": tname_db,
                    "col_id": c["col_id"],
                    "col_name_norm": col_norm_name,
                    "col_name_db": col_db_name,
                    "col_type": col_type,
                    "value": v_str,
                    "freq": freq,
                    "text": text,
                }
                cell_entries.append(entry)
                cell_texts.append(text)

        if (MAX_TOTAL_CELLS is not None) and (len(cell_entries) >= MAX_TOTAL_CELLS):
            break

    conn.close()

print("Cell-DB 엔트리 수:", len(cell_entries))

cell_emb = encoder.encode(
    cell_texts,
    normalize_embeddings=True,
    convert_to_numpy=True,
    show_progress_bar=True
)
print("Cell-DB embedding shape:", cell_emb.shape)

with open(CELL_INDEX_JSONL, "w", encoding="utf-8") as f:
    for e in cell_entries:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

np.save(CELL_EMB_NPY, cell_emb)
print("Cell-DB 저장 완료:", CELL_INDEX_JSONL, CELL_EMB_NPY)


Building Cell-DB (per DB):  75%|███████▍  | 124/166 [01:52<00:29,  1.40it/s]

[WARN] wta_1.players.first_name 쿼리 실패: Could not decode to UTF-8 column 'first_name' with text 'Selin G��Lseren'
[WARN] wta_1.players.last_name 쿼리 실패: Could not decode to UTF-8 column 'last_name' with text 'Treyes Albarrac��N'


Building Cell-DB (per DB): 100%|██████████| 166/166 [02:27<00:00,  1.12it/s]


Cell-DB 엔트리 수: 1121733


Batches:   0%|          | 0/35055 [00:00<?, ?it/s]

Cell-DB embedding shape: (1121733, 1024)
Cell-DB 저장 완료: /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/cell_index.jsonl /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed/cell_index_emb.npy


---
